## 결과 요약 : total_df 컬럼에 대한 PCA 분석
- 이 과정을 통해, KPI 선정 과정에서, 영향력있는 컬럼을 중심으로 데이터를 분석하고자 함.



- Order_id (16.12% 설명력): 주문 식별자는 데이터셋에서 가장 많은 정보를 포함하는 변수입니다. 주문 관련 다양한 요소들이 이 식별자와 연결되어 있을 수 있습니다.
- Order_item_id (15.28% 설명력): 각 주문에 포함된 아이템의 식별자 역시 상당한 양의 정보를 포함하고 있습니다. 주문된 아이템의 특성과 양이 이 변수를 통해 반영될 수 있습니다.
- Product_id (7.32% 설명력): 제품 식별자는 제품에 대한 중요한 정보를 포함합니다. 제품의 종류와 특성이 이 변수를 통해 데이터에 반영됩니다.
- Seller_id (5.82% 설명력): 판매자 식별자는 판매자의 특성과 판매 패턴을 반영할 수 있으며, 판매자의 다양성과 경쟁 상황을 나타냅니다.
- Price (4.81% 설명력): 제품의 가격은 구매 결정에 큰 영향을 미치며, 고객의 구매력과 제품의 가치를 반영합니다.
- Freight_value (4.22% 설명력): 배송료는 제품의 배송 조건과 비용을 나타내며, 지역적, 물류적 요인을 반영할 수 있습니다.
- Customer_id (4.14% 설명력): 고객 식별자는 특정 고객의 구매 행동과 패턴을 나타내며, 재구매, 고객 충성도 등에 영향을 미칠 수 있습니다.
- Order_purchase_timestamp (4.05% 설명력): 주문이 이루어진 시간은 시즌, 특정 이벤트, 경제 상황 등 다양한 시간적 요소를 반영할 수 있습니다.
- Order_delivered_customer_date (4.00% 설명력): 고객에게 배송 완료된 시간은 배송 속도와 효율성을 나타내며, 고객 만족도와 직접적인 관련이 있을 수 있습니다.
- Seller_zipcode_prefix (3.99% 설명력): 판매자의 지역 코드는 지리적 위치를 반영하며, 물류, 배송 네트워크의 효율성과 연결됩니다.


이 상위 10개 변수는 데이터의 69.75%의 분산을 설명하며, 이를 통해 데이터 세트의 대부분의 정보를 포착할 수 있습니다. 이 변수들은 데이터 분석 및 모델링에서 중요한 요소로 활용될 수 있습니다.

In [ ]:
# 필요한 라이브러리를 불러옵니다.
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# 데이터를 로드합니다.
data = pd.read_csv('./total_df.csv')

# Drop rows where review information is missing
data_cleaned = data.dropna(subset=['Review_score', 'Review_id', 'Review_creation_date', 'Review_answer_timestamp'])

# Convert product dimension and weight columns to numeric
dimension_cols = ['Product_weight_g', 'Product_length_cm', 'Product_height_cm', 'Product_width_cm']
data_cleaned[dimension_cols] = data_cleaned[dimension_cols].apply(pd.to_numeric, errors='coerce')

# Drop any rows that could not be converted (if any)
data_cleaned = data_cleaned.dropna(subset=dimension_cols)

# Convert dates from string to datetime
date_columns = ['Order_purchase_timestamp', 'Order_delivered_customer_date', 'Review_creation_date', 'Review_answer_timestamp']
for col in date_columns:
    data_cleaned[col] = pd.to_datetime(data_cleaned[col], errors='coerce')

# Calculate derived columns
# Delivery speed in days
data_cleaned['Delivery_speed_days'] = (data_cleaned['Order_delivered_customer_date'] - data_cleaned['Order_purchase_timestamp']).dt.days

# Review response time in days
data_cleaned['Review_response_time_days'] = (data_cleaned['Review_answer_timestamp'] - data_cleaned['Review_creation_date']).dt.days

# Display the new columns and any remaining missing values
derived_columns_info = data_cleaned[['Delivery_speed_days', 'Review_response_time_days']].describe()
missing_values_updated = data_cleaned.isnull().sum()
derived_columns_info, missing_values_updated


In [ ]:
data_cleaned.info()

In [ ]:
# Order_id 이름 바꾸기
data_cleaned['Order_id'] = data_cleaned['Order_id'].map(
    lambda x: x.replace('ORDER_', '')
)

data_cleaned.head()

In [ ]:
data_cleaned['Product_id'] = data_cleaned['Product_id'].map(
    lambda x: x.replace('PRODUCT_', '')
)

data_cleaned['Customer_id'] = data_cleaned['Customer_id'].map(
    lambda x: x.replace('CUSTOMER_', '')
)

data_cleaned['Review_id'] = data_cleaned['Review_id'].map(
    lambda x: x.replace('REVIEW_', '')
)

In [ ]:
data_cleaned['Seller_id'] = data_cleaned['Seller_id'].map(
    lambda x: x.replace('SELLER_', '')
)

In [ ]:
X = data_cleaned

# 데이터 표준화
scaler = StandardScaler()
X_scaled = pd.DataFrame()


for column in X.columns:
    try:
        # 특정 컬럼에 대해서만 fit_transform을 시도합니다.
        scaled_data = scaler.fit_transform(X[[column]])
        X_scaled[column] = scaled_data.flatten()  # 결과를 DataFrame에 추가합니다.
    except Exception as e:
        # 오류 발생시 해당 컬럼은 제외합니다.
        continue

# PCA 적용
pca = PCA()
pca_result = pca.fit_transform(X_scaled)

# 설명된 분산 비율
explained_variance = pca.explained_variance_ratio_
cumulative_variance = explained_variance.cumsum()




for i in range(0,25,1):
    print(f'{X_scaled.columns[i]}, : {explained_variance[i]}, {cumulative_variance[i]}')


#### 결과 요약
- Order_id (16.12% 설명력): 주문 식별자는 데이터셋에서 가장 많은 정보를 포함하는 변수입니다. 주문 관련 다양한 요소들이 이 식별자와 연결되어 있을 수 있습니다.
- Order_item_id (15.28% 설명력): 각 주문에 포함된 아이템의 식별자 역시 상당한 양의 정보를 포함하고 있습니다. 주문된 아이템의 특성과 양이 이 변수를 통해 반영될 수 있습니다.
- Product_id (7.32% 설명력): 제품 식별자는 제품에 대한 중요한 정보를 포함합니다. 제품의 종류와 특성이 이 변수를 통해 데이터에 반영됩니다.
- Seller_id (5.82% 설명력): 판매자 식별자는 판매자의 특성과 판매 패턴을 반영할 수 있으며, 판매자의 다양성과 경쟁 상황을 나타냅니다.
- Price (4.81% 설명력): 제품의 가격은 구매 결정에 큰 영향을 미치며, 고객의 구매력과 제품의 가치를 반영합니다.
- Freight_value (4.22% 설명력): 배송료는 제품의 배송 조건과 비용을 나타내며, 지역적, 물류적 요인을 반영할 수 있습니다.
- Customer_id (4.14% 설명력): 고객 식별자는 특정 고객의 구매 행동과 패턴을 나타내며, 재구매, 고객 충성도 등에 영향을 미칠 수 있습니다.
- Order_purchase_timestamp (4.05% 설명력): 주문이 이루어진 시간은 시즌, 특정 이벤트, 경제 상황 등 다양한 시간적 요소를 반영할 수 있습니다.
- Order_delivered_customer_date (4.00% 설명력): 고객에게 배송 완료된 시간은 배송 속도와 효율성을 나타내며, 고객 만족도와 직접적인 관련이 있을 수 있습니다.
- Seller_zipcode_prefix (3.99% 설명력): 판매자의 지역 코드는 지리적 위치를 반영하며, 물류, 배송 네트워크의 효율성과 연결됩니다.


이 상위 10개 변수는 데이터의 69.75%의 분산을 설명하며, 이를 통해 데이터 세트의 대부분의 정보를 포착할 수 있습니다. 이 변수들은 데이터 분석 및 모델링에서 중요한 요소로 활용될 수 있습니다.